In [14]:
import time


def one_task():
    print('begin task')
    print('  begin big_step: ')
    big_result = big_step()
    print(f'  end big_step with {big_result}')
    print('end_task')


def big_step():
    print('    begin small_step: ')
    small_result = small_step()
    print(f'    end small_step with {small_result}')
    return small_result * 1000


def small_step():
    print('      工作中')
    print('      歇会')
    time.sleep(2)
    print('      歇完了')

    return 666

In [15]:
# 执行简单版本
one_task()

begin task
  begin big_step: 
    begin small_step: 
      工作中
      歇会
      歇完了
    end small_step with 666
  end big_step with 666000
end_task


对 `small_step` 使用协程以避免阻塞, 顺便将函数名改为 `small_step_y`
将 `time.sleep(1)` 改为 `yield time.sleep(1)`

In [16]:
def small_step_y():
    print('      工作中')
    # yield time.sleep(1)
    # 自身无法消除阻塞, 将阻塞传给上游, 注: 此处 time.sleep, 1是作为元组传递的
    print('      歇会')
    yield time.sleep, 2
    print('      歇完了')
    return 666

由于 `yield` 具有传染性, 调用他的函数也需要做相应改变

In [17]:
def big_step_y():
    print('    begin small_step: ')
    # small_result = small_step()
    small_coro = small_step_y()
    while True:
        try:
            x = small_coro.send(None)
        except StopIteration as e:
            small_result = e.value
            break
        else:
            # 自己无法处理, 也传给上游
            yield x

    print(f'    end small_step with {small_result}')
    return small_result * 1000

这个函数是最上游, 需要对阻塞进行处理

In [18]:
def one_task_y():
    print('begin task')
    print('  begin big_step: ')
    # big_result = big_step()
    big_coro = big_step_y()
    while True:
        try:
            x = big_coro.send(None)
        except StopIteration as e:
            big_result = e.value
            break
        else:
            # 处理阻塞
            func, arg = x
            func(arg)
    print(f'  end big_step with {big_result}')
    print('end_task')

In [19]:
one_task_y()

begin task
  begin big_step: 
    begin small_step: 
      工作中
      歇会
      歇完了
    end small_step with 666
  end big_step with 666000
end_task


# yield from 一统天下
使用 yield from 来优化代码

In [20]:
class YieldFromAble:
    def __init__(self, obj):
        self.value = obj

    def __iter__(self):
        yield self


def small_step_yf():
    print('      工作中')
    # yield time.sleep(1)
    # 自身无法消除阻塞, 将阻塞传给上游, 注: 此处 time.sleep, 2是作为元组传递的
    print('      歇会')
    # 为了防止元组被当作可迭代对象来创造成一个生成器, 借助工具类YieldFromAble将元组统一进行返回
    yield from YieldFromAble((time.sleep, 2))
    print('      歇完了')
    return 666


def big_step_yf():
    print('    begin small_step: ')

    small_result = yield from small_step_yf()

    print(f'    end small_step with {small_result}')
    return small_result * 1000


def one_task_yf():
    print('begin task')
    print('  begin big_step: ')

    big_result = yield from big_step_yf()

    print(f'  end big_step with {big_result}')
    print('end_task')

### 此时one_task_yf变成了一个新的生成器
 完善代码, 需要创建一个任务驱动器

In [21]:
class TaskDrive:
    def __init__(self, coro):
        self.coro = coro

    def run(self):
        print('--' * 10)
        while True:
            try:
                x = self.coro.send(None)
            except StopIteration as e:
                result = e.value
                break
            else:
                # 处理阻塞
                func, arg = x.value
                func(arg)
        print('--' * 10)

In [22]:
t = TaskDrive(one_task_yf())
t.run()

--------------------
begin task
  begin big_step: 
    begin small_step: 
      工作中
      歇会
      歇完了
    end small_step with 666
  end big_step with 666000
end_task
--------------------
